In [2]:
import matlab.engine

# Test functions
Code samples taken from https://www.mathworks.com/help/matlab/matlab_external/call-matlab-functions-from-python.html

In [3]:
eng = matlab.engine.start_matlab()

In [4]:
tf = eng.isprime(37)

In [5]:
print(tf)

True


In [6]:
t = eng.gcd(100.0,80.0,nargout=3)
t

(20.0, 1.0, -1.0)

In [7]:
eng.doc(nargout=0)

In [11]:
cd ../notebooks/

/Users/morganschwartz/Code/germband-extension/notebooks


In [12]:
eng.triarea(nargout=0)

Need to add matlab folder containing scripts to matlab path in order to have the scripts be locatable.

In [13]:
eng.addpath(r'../matlab',nargout=0)

In [ ]:
eng.triarea(nargout=0)

# Test OpticalFlow function

[Documentation for OpticalFlow](/Users/morganschwartz/Zotero/storage/5ER2GAAS/mmc1.pdf )

## Define parameters

In [33]:
MovieName = '/Users/morganschwartz/Code/germband-extension/data/wt_gbe_20180110_mask.tif'
BinaryMask = matlab.single([])
scale = 0.5
dt = 1.0
BoxSize = 30
BlurSTD = 1.0 # Must be float
ArrowSize= 5

In [29]:
matlab.int8([])

matlab.int8([])

`OpticalFlow` has 5 outputs

In [42]:
out = eng.OpticalFlow(MovieName,BinaryMask,BoxSize,BlurSTD,ArrowSize,scale,dt,nargout=5)

ValueError: only a scalar struct can be returned from MATLAB

Using `nargout` seems to produce an error. Output data types aren't compatible with engine passing to python. Would it be better to save to a file?

In [36]:
type(out)

mlarray.double

The matlab function `csvwrite` saves arrays to `.csv` files. I will try to modify `OpticalFlow` to write data outputs to files.

# Test output function

In [52]:
eng.OpticalFlowOutput('test2',MovieName,BinaryMask,BoxSize,BlurSTD,ArrowSize,scale,dt,nargout=0)

Succesfully saved output data to csv files. It seems like they save in whatever the working directory is for the notebook.

# Read in test output files

In [45]:
import numpy as np
import pandas as pd

In [53]:
rawx = pd.read_csv('test2_X.csv',header=None)
rawx.head()
x = np.array(rawx)
x.shape

(40401, 1)

Based on running this function in matlab, I would only expect ~2600 rows.

In [54]:
rawvx = pd.read_csv('test2_Vx.csv',header=None)
vx = np.array(rawvx)
vx.shape

(40401, 166)

In [51]:
rawvx.head()

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
0,0,0.026191,-0.031664,0.020631,-0.052106,-0.020458,0.006562,-0.037577,0.023255,0.005738,...,-0.645180,1.196100,0.403000,0.134300,-1.575700,1.264700,1.344400,-0.522450,-1.410900,1.196300
1,0,0.003324,-0.007919,0.000586,-0.004572,-0.003323,0.003327,-0.007001,0.003221,0.004370,...,-0.021795,0.040757,-0.004139,0.019235,-0.046742,0.045661,0.014661,-0.031069,-0.040504,0.035840
2,0,0.003037,-0.007645,0.000434,-0.004055,-0.003170,0.003270,-0.006595,0.002929,0.004490,...,-0.017896,0.033684,-0.009703,0.018522,-0.033342,0.036769,-0.006532,-0.028783,-0.029349,0.026410
3,0,0.002984,-0.007658,0.000492,-0.004084,-0.003207,0.003233,-0.006631,0.002929,0.004623,...,-0.018905,0.036036,-0.010215,0.019621,-0.035047,0.039157,-0.006001,-0.031385,-0.030332,0.027346
4,0,0.002930,-0.007673,0.000552,-0.004116,-0.003245,0.003195,-0.006670,0.002932,0.004763,...,-0.020041,0.038750,-0.010755,0.020953,-0.037020,0.041926,-0.004557,-0.034407,-0.031414,0.028379


In [55]:
40401/2601 

15.53287197231834

In [60]:
vx2 = np.genfromtxt('test2_Vx.csv')
vx.shape

(40401, 166)

The large number of rows does not appear to be a pandas related error.

From https://www.mathworks.com/help/matlab/matlab_external/redirect-standard-output-and-error-to-python.html
```
import matlab.engine
eng = matlab.engine.start_matlab()
import io
out = io.StringIO()
err = io.StringIO()
ret = eng.dec2base(2**60,16,stdout=out,stderr=err)```

In [61]:
import io

In [62]:
out = io.StringIO()
err = io.StringIO()

In [63]:
eng.OpticalFlowOutput('test2',MovieName,BinaryMask,BoxSize,BlurSTD,ArrowSize,scale,dt,nargout=0,stdout=out,stderr=err)

In [66]:
print(out.getvalue())

       40401           1

       40401           1

       40401         166

       40401         166




In [65]:
print(err.getvalue())

It seems like the number of rows may be correct...? The printout of array size from within Matlab is the same as the number of rows in the array after import to python.

# Tidy data

Files `*_X.csv` and `*_Y.csv` contain the x and y positions of the velocity vectors. Files `*_Vx.csv` and `*_Vy.csv` are the velocity vectors in the form of a MxN matrix where M is the number of velocity vectors and N is the number of timepoints.

In [67]:
x = pd.read_csv('test2_X.csv',header=None)
y = pd.read_csv('test2_Y.csv',header=None)
vx = pd.read_csv('test2_Vx.csv',header=None)
vy = pd.read_csv('test2_Vy.csv',header=None)

In [85]:
xy = pd.concat([x.rename(columns={0:'x'}),
                y.rename(columns={0:'y'})
                ],axis=1)
xy.head()

,x,y
0,11,11
1,11,16
2,11,21
3,11,26
4,11,31


In [84]:
vx.melt().rename(columns={'variable':'frame','value':'vx'})

,frame,vx
0,0,0.000000
1,0,0.000000
2,0,0.000000
3,0,0.000000
4,0,0.000000
5,0,0.000000
6,0,0.000000
7,0,0.000000
8,0,0.000000
9,0,0.000000


In [90]:
vxvy = pd.concat([vx.melt().rename(columns={'value':'vx'}).drop(columns=['variable']),
                  vy.melt().rename(columns={'variable':'frame','value':'vy'})
                 ],axis=1)

In [92]:
vxvy.head()

,vx,frame,vy
0,0.0,0,0.0
1,0.0,0,0.0
2,0.0,0,0.0
3,0.0,0,0.0
4,0.0,0,0.0


In [95]:
vectors = vxvy.merge(xy,left_on='frame',right_index=True)
vectors.head()

,vx,frame,vy,x,y
0,0.0,0,0.0,11,11
1,0.0,0,0.0,11,11
2,0.0,0,0.0,11,11
3,0.0,0,0.0,11,11
4,0.0,0,0.0,11,11


The DataFrame `vectors` is in an appropriate tidy format, but unfortunately functions that can plot quiver plots do not accept pandas dataframes as an input. 

Data need to be cast into the following form:
>x and y are 1d arrays defining an *evenly spaced* grid.  
u and v are 2d arrays (shape [y,x]) giving velocities.

x and y can be created using `np.meshgrid` with the unique values in `vectors['x']` and `vectors['y']`

## Test tidy dataframe utility function

In [2]:
from imp import reload
import gbeflow

In [6]:
reload(gbeflow)

<module 'gbeflow' from '/Users/morganschwartz/Code/germband-extension/gbeflow/__init__.py'>

In [7]:
vxvy = gbeflow.tidy_vector_data('test2')

In [8]:
vxvy.head()

,vx,frame,vy,x,y
0,0.0,0,0.0,11,11
1,0.0,0,0.0,11,11
2,0.0,0,0.0,11,11
3,0.0,0,0.0,11,11
4,0.0,0,0.0,11,11
